# This book generates an RNN model on weekly soybean

### CONTENTS
### 1 Imports
### 2 Arrange datasets
### 3 Define models
### 4 Train models

# 1 Imports

In [ ]:
# RNN imports

import tensorflow as tf

from keras.preprocessing.sequence import TimeseriesGenerator

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN

from keras.models import load_model

from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
#General imports

import numpy as np
import pandas as pd

import scipy.stats as stats

import matplotlib.pyplot as plt
import matplotlib.dates

from datetime import datetime

from google.colab import drive
import json

In [ ]:
# Import weekly data

data_url = 'https://raw.githubusercontent.com/kumars8/Deloitte_login_prjct/master/soybean_weekly.csv'
soybean_df = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='date')

# fix bad name
soybean_df.columns = ['value']

# take only 1974 - 2017
soybean_df = soybean_df['1974-01-01':'2017-12-31']

# soybean_df needs to lose last value becuase during process of VMD, the last value is lost
soybean_df = soybean_df[:-1]

# view loaded data
soybean_df.tail(2)

,value
date,
2017-12-10,997.8
2017-12-17,974.3


In [ ]:
# Import emd data
data_url = 'https://raw.githubusercontent.com/kumars8/Deloitte_login_prjct/master/soybean_emd_weekly.csv'
soybean_emd = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='date')

In [ ]:
# Import vmd data
data_url = 'https://raw.githubusercontent.com/kumars8/Deloitte_login_prjct/master/soybean_vmd_weekly.csv'
soybean_vmd = pd.read_csv(data_url, parse_dates=True, dayfirst=True, index_col='date')

In [ ]:
# Check indices are aligned
print(soybean_df.index.equals(soybean_emd.index))
print(soybean_df.index.equals(soybean_vmd.index))

True
True


# 2 Arrange datasets

In [ ]:
# Split data so that valid and test do not feature in decisions
from sklearn.model_selection import train_test_split

true_train, out_of_sample = train_test_split(soybean_df, test_size=0.2, shuffle=False)
true_valid, true_test = train_test_split(out_of_sample, test_size=0.5, shuffle=False)

emd_train, out_of_sample = train_test_split(soybean_emd, test_size=0.2, shuffle=False)
emd_valid, emd_test = train_test_split(out_of_sample, test_size=0.5, shuffle=False)

vmd_train, out_of_sample = train_test_split(soybean_vmd, test_size=0.2, shuffle=False)
vmd_valid, vmd_test = train_test_split(out_of_sample, test_size=0.5, shuffle=False)

In [ ]:
# Scale data between 0 and 1
from sklearn.preprocessing import MinMaxScaler

emd_scaler = MinMaxScaler()
emd_train = emd_scaler.fit_transform(emd_train)
emd_valid = emd_scaler.transform(emd_valid)

vmd_scaler = MinMaxScaler()
vmd_train = vmd_scaler.fit_transform(vmd_train)
vmd_valid = vmd_scaler.transform(vmd_valid)

true_scaler = MinMaxScaler()
true_train = true_scaler.fit_transform(true_train)
true_valid = true_scaler.transform(true_valid)

In [ ]:
# Set input shape variables
emd_n_inputs = 4
vmd_n_inputs = 5
non_decomp_n_inputs = 5

In [ ]:
# get emd sets in order
emd_extra_valid = emd_train[-emd_n_inputs:]
emd_valid_set = np.concatenate((emd_extra_valid, emd_valid))

emd_extra_test = emd_valid[-emd_n_inputs:]
emd_test_set = np.concatenate((emd_extra_test, emd_test))

emd_train_shaped = TimeseriesGenerator(emd_train, emd_train, length=emd_n_inputs, batch_size=1)
emd_valid_shaped = TimeseriesGenerator(emd_valid_set, emd_valid_set, length=emd_n_inputs, batch_size=1)
emd_test_shaped = TimeseriesGenerator(emd_test_set, emd_test_set, length=emd_n_inputs, batch_size=1)

In [ ]:
# get vmd sets in order
vmd_extra_valid = vmd_train[-vmd_n_inputs:]
vmd_valid_set = np.concatenate((vmd_extra_valid, vmd_valid))

vmd_extra_test = vmd_valid[-vmd_n_inputs:]
vmd_test_set = np.concatenate((vmd_extra_test, vmd_test))

vmd_train_shaped = TimeseriesGenerator(vmd_train, vmd_train, length=vmd_n_inputs, batch_size=1)
vmd_valid_shaped = TimeseriesGenerator(vmd_valid_set, vmd_valid_set, length=vmd_n_inputs, batch_size=1)
vmd_test_shaped = TimeseriesGenerator(vmd_test_set, vmd_test_set, length=vmd_n_inputs, batch_size=1)

In [ ]:
# get non-decomposed sets in order
extra_valid = true_train[-non_decomp_n_inputs:]
valid_set = np.concatenate((extra_valid, true_valid))

extra_test = true_valid[-non_decomp_n_inputs:]
test_set = np.concatenate((extra_test, true_test))

train_shaped = TimeseriesGenerator(true_train, true_train, length=non_decomp_n_inputs, batch_size=1)
valid_shaped = TimeseriesGenerator(valid_set, valid_set, length=non_decomp_n_inputs, batch_size=1)
test_shaped = TimeseriesGenerator(test_set, test_set, length=non_decomp_n_inputs, batch_size=1)

# 3 Define models

In [ ]:
# Set shape variables
emd_units = 7

vmd_L1_units = 7
vmd_L2_units = 4

non_decomp_units = 7

# Set number of features
n_features = 1

In [ ]:
# Set other variables
activation = 'relu'
dropout = 0
optimizer = 'adam'
loss = 'mse'

In [ ]:
# Define emd model definition function

def define_emd_model():
  RNNregressor = Sequential()
  RNNregressor.add(SimpleRNN(units=emd_units, input_shape=(emd_n_inputs, n_features), activation=activation, dropout=dropout))
  RNNregressor.add(Dense(units=1))
  RNNregressor.compile(optimizer=optimizer, loss=loss)
  return RNNregressor

In [ ]:
# Define vmd model definition function

def define_vmd_model():
  RNNregressor = Sequential()
  RNNregressor.add(SimpleRNN(units=vmd_L1_units, input_shape=(vmd_n_inputs, n_features), activation=activation, dropout=dropout, return_sequences=True))
  RNNregressor.add(SimpleRNN(units=vmd_L2_units, input_shape=(vmd_n_inputs, n_features), activation=activation, dropout=dropout))
  RNNregressor.add(Dense(units=1))
  RNNregressor.compile(optimizer=optimizer, loss=loss)
  return RNNregressor

In [ ]:
# Define non decomp model definition function

def define_non_decomp_model():
  RNNregressor = Sequential()
  RNNregressor.add(SimpleRNN(units=non_decomp_units, input_shape=(non_decomp_n_inputs, n_features), activation=activation, dropout=dropout))
  RNNregressor.add(Dense(units=1))
  RNNregressor.compile(optimizer=optimizer, loss=loss)
  return RNNregressor

# 4 Train models

In [ ]:
# Set variables

decomp_EPOCHS = 50
non_decomp_EPOCHS = 8

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###on EMD soybean weekly

In [ ]:
# Set checkpoint callback
filepath = 'emd_RNN_weekly_valid_{epoch:02d}'
checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'))

# Define model
EMDregressor = define_emd_model()

# Train model
with tf.device('/device:GPU:0'):
  EMDregressor.fit(emd_train_shaped,
                   epochs=decomp_EPOCHS,
                   validation_data=emd_valid_shaped,
                   callbacks=[checkpoint])

Epoch 1/50
1831/1831 [==============================] - 14s 6ms/step - loss: 0.0044 - val_loss: 0.0028
Epoch 2/50
1831/1831 [==============================] - 11s 6ms/step - loss: 2.6855e-04 - val_loss: 0.0011
Epoch 3/50
1831/1831 [==============================] - 12s 6ms/step - loss: 2.2083e-04 - val_loss: 4.8752e-04
Epoch 4/50
1831/1831 [==============================] - 13s 7ms/step - loss: 2.0783e-04 - val_loss: 2.6398e-04
Epoch 5/50
1831/1831 [==============================] - 12s 6ms/step - loss: 1.6371e-04 - val_loss: 4.5069e-04
Epoch 6/50
1831/1831 [==============================] - 12s 6ms/step - loss: 1.3749e-04 - val_loss: 2.6991e-04
Epoch 7/50
1831/1831 [==============================] - 11s 6ms/step - loss: 1.2520e-04 - val_loss: 4.0164e-04
Epoch 8/50
1831/1831 [==============================] - 11s 6ms/step - loss: 1.1200e-04 - val_loss: 2.1867e-04
Epoch 9/50
1831/1831 [==============================] - 11s 6ms/step - loss: 1.0606e-04 - val_loss: 2.2629e-04
Epoch 10/50
1

In [ ]:
## ACTION REQUIRED ##

# Select best model by lowest validation loss
best_model = 'emd_RNN_weekly_valid_46'

In [ ]:
# Save best model to Drive
emd_RNN_weekly_model = load_model(f'models/{best_model}.model')
emd_RNN_weekly_model.save(f'/content/gdrive/My Drive/models1/{best_model}.h5')

In [ ]:
# Use best model to generate emd validation set forecasts
emd_fc = emd_RNN_weekly_model.predict(emd_valid_shaped)

In [ ]:
# Rescale forecasted values
emd_rescaled = emd_scaler.inverse_transform(emd_fc)

In [ ]:
# Save emd validation set predictions
np.save('/content/gdrive/MyDrive/weekly_soybean/emd_RNN_weekly_valid.npy', emd_rescaled.flatten())

###on VMD soybean weekly

In [ ]:
# Set checkpoint callback
filepath = 'vmd_RNN_weekly_valid_{epoch:02d}'
checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'))

# Define model
VMDregressor = define_vmd_model()

# Train model
with tf.device('/device:GPU:0'):
  VMDregressor.fit(vmd_train_shaped,
                   epochs=decomp_EPOCHS,
                   validation_data=vmd_valid_shaped,
                   callbacks=[checkpoint])

Epoch 1/50
1830/1830 [==============================] - 25s 13ms/step - loss: 0.0023 - val_loss: 0.0014
Epoch 2/50
1830/1830 [==============================] - 23s 12ms/step - loss: 6.8946e-04 - val_loss: 0.0014
Epoch 3/50
1830/1830 [==============================] - 22s 12ms/step - loss: 4.5642e-04 - val_loss: 5.6243e-04
Epoch 4/50
1830/1830 [==============================] - 23s 12ms/step - loss: 3.5810e-04 - val_loss: 8.7811e-04
Epoch 5/50
1830/1830 [==============================] - 23s 12ms/step - loss: 2.8464e-04 - val_loss: 5.1714e-04
Epoch 6/50
1830/1830 [==============================] - 23s 12ms/step - loss: 2.5458e-04 - val_loss: 3.6788e-04
Epoch 7/50
1830/1830 [==============================] - 23s 12ms/step - loss: 1.9343e-04 - val_loss: 3.5588e-04
Epoch 8/50
1830/1830 [==============================] - 22s 12ms/step - loss: 1.8025e-04 - val_loss: 8.1889e-04
Epoch 9/50
1830/1830 [==============================] - 23s 13ms/step - loss: 1.4130e-04 - val_loss: 1.8100e-04
Epoc

In [ ]:
## ACTION REQUIRED ##

# Select best model by lowest validation loss
best_model = 'vmd_RNN_weekly_valid_38'

In [ ]:
# Save best model to Drive
vmd_RNN_weekly_model = load_model(f'models/{best_model}.model')
vmd_RNN_weekly_model.save(f'/content/gdrive/MyDrive/models1/{best_model}.h5')

In [ ]:
# Use best model to generate emd validation set forecasts
vmd_fc = vmd_RNN_weekly_model.predict(vmd_valid_shaped)

In [ ]:
# Rescale forecasted values
vmd_rescaled = vmd_scaler.inverse_transform(vmd_fc)

In [ ]:
# Save emd validation set predictions
np.save('/content/gdrive/MyDrive/weekly_soybean/vmd_RNN_weekly_valid.npy', vmd_rescaled.flatten())

## on non-decomposed

In [ ]:
# Set checkpoint callback
filepath = 'true_RNN_weekly_valid_{epoch:02d}'
checkpoint = ModelCheckpoint('models/{}.model'.format(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min'))

# Define model
TRUEregressor = define_non_decomp_model()

# Train model
with tf.device('/device:GPU:0'):
  TRUEregressor.fit(train_shaped,
                   epochs=non_decomp_EPOCHS,
                   validation_data=valid_shaped,
                   callbacks=[checkpoint])

Epoch 1/8
1830/1830 [==============================] - 13s 7ms/step - loss: 0.0015 - val_loss: 0.0033
Epoch 2/8
1830/1830 [==============================] - 12s 7ms/step - loss: 7.5575e-04 - val_loss: 0.0033
Epoch 3/8
1830/1830 [==============================] - 12s 7ms/step - loss: 5.9778e-04 - val_loss: 0.0024
Epoch 4/8
1830/1830 [==============================] - 13s 7ms/step - loss: 5.1483e-04 - val_loss: 0.0011
Epoch 5/8
1830/1830 [==============================] - 12s 7ms/step - loss: 4.4956e-04 - val_loss: 0.0018
Epoch 6/8
1830/1830 [==============================] - 12s 7ms/step - loss: 4.4082e-04 - val_loss: 0.0011
Epoch 7/8
1830/1830 [==============================] - 12s 7ms/step - loss: 4.1862e-04 - val_loss: 0.0013
Epoch 8/8
1830/1830 [==============================] - 13s 7ms/step - loss: 3.9479e-04 - val_loss: 9.9047e-04


In [ ]:
## ACTION REQUIRED ##

# Select best model by lowest validation loss
best_model = 'true_RNN_weekly_valid_05'

In [ ]:
# Save best model to Drive
true_RNN_weekly_model = load_model(f'models/{best_model}.model')
true_RNN_weekly_model.save(f'/content/gdrive/MyDrive/models1/{best_model}.h5')

In [ ]:
# Use best model to generate emd validation set forecasts
true_fc = true_RNN_weekly_model.predict(valid_shaped)

In [ ]:
# Rescale forecasted values
true_rescaled = true_scaler.inverse_transform(true_fc)

In [ ]:
# Save emd validation set predictions
np.save('/content/gdrive/MyDrive/weekly_soybean/true_RNN_weekly_valid.npy', true_rescaled.flatten())

# End